[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/allegheny-college-cmpsc-101-fall-2024/course-materials/blob/main/code/20241125_optimization.ipynb)

In [ ]:
# TODO fill in eight food names, values, and calories as the initial data for this problem

food_names = ['', '', '', '', '', '', '', '', '']
values = [,,,,,,,,]
calories = [,,,,,,,,]

calorie_cap = # TODO

## Greedy code

In [ ]:
# Define the class that will represent the data values

from typing import List
from typing import Tuple


class Food(object):

    def __init__(self, name_of_food: str, value_deliciousness: int, cost_in_calories: int):
        """Construct an instance of the Food class."""
        self._name = name_of_food
        self._value = value_deliciousness
        self._calories = cost_in_calories

    def get_name(self) -> str:
        """Access the name of the Food object."""
        return self._name

    def get_value(self) -> int:
        """Access how deliciously valuable the Food object is."""
        return self._value

    def get_cals(self) -> int:
        """Access the caloric cost of the Food object."""
        return self._calories

    def density(self) -> float:
        """Access the value to calorie ratio."""
        return self.get_value()/self.get_cals()

    def __repr__(self) -> str:
        """Produce a textual representation of the Food."""
        return f"(One {self._name} with {self._calories} calories is valued at {self._value} units )"

In [ ]:
# define a function that can creates food instances for the 0/1 Knapsack problem

def build_menu(food_names: List[str], values: List[int], calories: List[int]) -> List[Food]:
    """Create an instance of a 0/1 knapsack using instances of the Item class."""
    menu: List[Food] = []
    for f in range(len(food_names)):
        menu.append(Food(food_names[f], values[f], calories[f]))
    return menu

In [ ]:
# define the helper functions that work with an instance of the class to get useful numbers

def order_by_value(food: Food) -> int:
    """Return the value for a specific item."""
    return food.get_value()


def order_by_calories(food: Food) -> float:
    """Return the inverse of the weight for a specific item."""
    return 1.0 / food.get_cals()


def order_by_density(food: Food) -> float:
    """Return the density of the item."""
    return food.get_value() / food.get_cals()

In [ ]:
# create a greedy solver

def greedy(menu: List[Food], calorie_cap: int, order_function) -> Tuple[List[Food], float, float]:
    """Perform the greedy algorithm for items, a maximum weight of a knapsack, and an objective function."""
    menu_sorted = sorted(menu, key=order_function, reverse=True) # descending order
    selected_foods: List[Food] = []
    value_so_far = 0.0
    calories_so_far = 0.0
    for i in range(len(menu_sorted)):
        if (calories_so_far + menu_sorted[i].get_cals()) <= calorie_cap:
            selected_foods.append(menu_sorted[i])
            calories_so_far += menu_sorted[i].get_cals()
            value_so_far += menu_sorted[i].get_value()
    return (selected_foods, value_so_far, calories_so_far)

In [ ]:
# define the functions for running the greedy algorithm

def run_and_display_one_greedy(menu: List[Food], calorie_cap: int, order_function) -> None:
    """Run the greedy algorithm and display the result."""
    selected_foods, value, cals = greedy(menu, calorie_cap, order_function)
    print("Total value of selected foods is", value)
    print("Total calories in selected foods is", cals)
    for food in selected_foods:
        print("  ", food)

def run_all_greedy(menu: List[Food], calorie_cap=1000) -> None:
    """Run all greedy algorithm with all possible objective functions."""
    print("Running all of the knapsack solvers!")
    print()
    print(f"Using greedy-by-value to select foods up to {calorie_cap} calories")
    run_and_display_one_greedy(menu, calorie_cap, order_by_value)
    print()
    print(f"Using greedy-by-calories to select foods up to {calorie_cap} calories")
    run_and_display_one_greedy(menu, calorie_cap, order_by_calories)
    print()
    print(f"Using greedy-by-density to select foods up to {calorie_cap} calories")
    run_and_display_one_greedy(menu, calorie_cap, order_by_density)
    print()

In [ ]:
menu = build_menu(food_names, values, calories)
run_all_greedy(menu, calorie_cap)

## Powerset code

In [ ]:
def powerset(s: List[int]):
    """Generate the powerset of a list of items."""
    # Reference:
    # https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    # https://realpython.com/python-bitwise-operators/
    # https://docs.python.org/3.11/library/functions.html#zip
    x = len(s)
    masks = [1 << i for i in range(x)] # list of shifted ones
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]



def exhaustive_enumeration(pset, max_weight, get_value, get_weight):
    """Run an exhaustive enumeration algorithm to find best combination."""
    best_value = 0.0
    best_set = []
    best_cals = 0.0
    for foods in pset:
        foods_value = 0.0
        total_cals = 0.0
        for f in foods:
            foods_value += get_value(f)
            total_cals += get_weight(f)
        if total_cals <= max_weight and foods_value > best_value:
            best_value = foods_value
            best_set = foods
            best_cals = total_cals
    return (best_set, best_value, best_cals)


In [ ]:
# define the function for running the exhaustive algorithm

def run_exhaustive_enumeration(menu, calorie_cap=1000):
    """Use the exhaustive enumeration algorithm for a problem instance."""
    print("Generating the powerset of all items!")
    pset = powerset(menu)
    print()
    print("Using exhaustive enumeration to fill a knapsack of size", calorie_cap)
    taken, value, cals = exhaustive_enumeration(pset, calorie_cap, Food.get_value, Food.get_cals)
    print("Total value of selected foods is", value)
    print("Total calories in selected foods are", cals)
    for item in taken:
        print("  ", item)

In [ ]:
# run the exhaustive algorithm for the specific instance and display the solution

menu = build_menu(food_names, values, calories)
run_exhaustive_enumeration(menu, calorie_cap)

In [ ]:
# Questions:
# 1) What are the similarities and differences between the greedy and exhaustive algorithms?
# 2) Which algorithm is likely to be faster, the greedy or the exhaustive one?
# 3) Which algorithm is likely to produce a better answer, the greedy or the exhaustive one?


In [ ]:
# define your own instance of the 0/1 knapsack and solve it with a greedy algorithm

In [ ]:
# define your own instance of the 0/1 knapsack and solve it with the exhaustive algorithm